In [46]:
import yaml
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import inspect
from sqlalchemy import text
from pandasgui import show
import numpy as np 
import tabula
import requests
import boto3


In [47]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning

In [48]:
yaml_file_path = 'db_creds.yaml'
RDS_CONNECTOR = DatabaseConnector(yaml_file_path)

RDS_CONNECTOR.init_db_engine()
       
Display_Data = DataExtractor(RDS_CONNECTOR.engine)

store_details_df = Display_Data.reads_rds_table("legacy_store_details")
user_data_df = Display_Data.reads_rds_table("legacy_users")
orders_df = Display_Data.reads_rds_table("orders_table")

pdf_url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
card_details_df = Display_Data.retrieve_pdf_data(pdf_url)


number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
retrieve_store_endpoint = 'store_details'

number_of_stores = Display_Data.list_number_of_stores(number_of_stores_endpoint)
stores_df = Display_Data.retrieve_stores_data(retrieve_store_endpoint, number_of_stores)

s3_address = 's3://data-handling-public/products.csv'
product_df = Display_Data.extract_from_s3(s3_address)

date_events_url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'
date_events_df = Display_Data.extract_date_events_data(date_events_url)

# print(date_events_df.to_markdown())


c:\Users\holme\miniconda3\envs\mrdc\lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


{'statusCode': 200, 'number_stores': 451}


In [49]:
yaml_file_path2 = 'local_db_cred.yaml'
LOCAL_CONNECTOR = DatabaseConnector(yaml_file_path2)

LOCAL_CONNECTOR.init_db_engine()

clean_user_data = DataCleaning(user_data_df)
cleaned_user_data = clean_user_data.clean_legacy_users()
LOCAL_CONNECTOR.upload_to_db(cleaned_user_data, 'dim_users')

clean_orders_data = DataCleaning(orders_df)
cleaned_orders_data = clean_orders_data.clean_orders_data()
LOCAL_CONNECTOR.upload_to_db(cleaned_orders_data, 'orders_table')

clean_card_details_data = DataCleaning(card_details_df)
cleaned_card_details_data = clean_card_details_data.clean_card_data()
LOCAL_CONNECTOR.upload_to_db(cleaned_card_details_data, 'dim_card_details')

clean_stores_data = DataCleaning(stores_df)
cleaned_stores_data = clean_stores_data.clean_store_data()
LOCAL_CONNECTOR.upload_to_db(cleaned_stores_data, 'dim_store_details')

clean_product_data = DataCleaning(product_df)
cleaned_product_data = clean_product_data.convert_product_weights()
cleaned_product_data = clean_product_data.clean_products_data()
LOCAL_CONNECTOR.upload_to_db(cleaned_product_data, 'dim_products')

clean_date_events_data = DataCleaning(date_events_df)
cleaned_date_events_data = clean_date_events_data.clean_data_events()
LOCAL_CONNECTOR.upload_to_db(cleaned_date_events_data, 'dim_dates_times')

InternalError: (psycopg2.errors.ReadOnlySqlTransaction) cannot execute CREATE TABLE in a read-only transaction

[SQL: 
CREATE TABLE dim_users (
	first_name TEXT, 
	last_name TEXT, 
	date_of_birth DATE, 
	company TEXT, 
	email_address TEXT, 
	address TEXT, 
	country TEXT, 
	country_code TEXT, 
	phone_number TEXT, 
	join_date DATE, 
	user_uuid TEXT
)

]
(Background on this error at: https://sqlalche.me/e/20/2j85)